## 1. Setup: Import libraries and load environment

As in previous labs, you now import the required libraries, load environment variables, and set up helper utilities.

In [2]:
# =========================
# Imports
# =========================

# --- Standard library ---
import base64
import json
import os
import re
from datetime import datetime
from io import BytesIO

# --- Third-party ---
import requests
import openai
from PIL import Image
from dotenv import load_dotenv
from IPython.display import Markdown, display
import aisuite

# --- Local / project ---
import tools
import utils


# =========================
# Environment & Client
# =========================
load_dotenv()
client = aisuite.Client()


## 2. My Agent  

My goal is to create a simple marketing campaign agent from the future, featuring an Art Director and a Copywriter who reviews the text and creates marketing hooks. I also want it to include the date the report was generated: 

In [5]:
MODEL_ID = "openai:gpt-4o"
PACKAGING_MODEL = "openai:o4-mini"

# 1. CORE AGENT FUNCTION
def run_agent(role_prompt, user_input, is_json=False):
    messages = [
        {"role": "system", "content": role_prompt},
        {"role": "user", "content": user_input}
    ]
    
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=messages,
        response_format={"type": "json_object"} if is_json else None
    )
    
    content = response.choices[0].message.content
    return json.loads(content) if is_json else content

# 2. AGENT DEFINITIONS

def trend_researcher_agent(topic):
    """Acts as the Creative/Trend analyst."""
    prompt = """
    You are a Futuristic Fashion Trend Analyst. 
    Analyze the given topic and provide a detailed summary.
    Return a JSON with:
    - 'trend_summary': A detailed analysis of materials and aesthetics.
    - 'visual_description': A prompt description for an image.
    """
    return run_agent(prompt, topic, is_json=True)

def copywriter_agent(trend_data):
    """Acts as the Marketing Copywriter."""
    prompt = f"""
    You are an Expert Copywriter. Based on these trends: {trend_data['trend_summary']},
    create a powerful marketing quote and a justification.
    Return a JSON with:
    - 'quote': A catchy, futuristic slogan.
    - 'justification': Why this resonates with the target audience.
    """
    return run_agent(prompt, str(trend_data), is_json=True)

# 3. YOUR SPECIFIC PACKAGING AGENT

def packaging_agent(trend_summary: str, image_url: str, quote: str, justification: str, output_path: str = "campaign_summary.md") -> str:
    styled_image_html = f"![Campaign Visual]({image_url})"

    print("Packaging Agent rewriting for executives...")
    beautified_summary = client.chat.completions.create(
        model=PACKAGING_MODEL,
        messages=[
            {"role": "system", "content": "You are a marketing communication expert writing elegant campaign summaries for executives."},
            {"role": "user", "content": f"Please rewrite the following trend summary to be clear, professional, and engaging for a CEO audience:\"\"\"{trend_summary.strip()}\"\"\""}
        ]
    ).choices[0].message.content.strip()

    # markdown content construction
    markdown_content = f"""# Futuristic Clothing Campaign – Executive Summary

## Refined Trend Insights
{beautified_summary}

## Campaign Visual
{styled_image_html}

## Campaign Quote
> {quote.strip()}

## Why This Works
{justification.strip()}

---
*Report generated on {datetime.now().strftime('%Y-%m-%d')}*"""

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)

    return output_path

# Step 1: Research
market_research_result = trend_researcher_agent("Sustainable Cyberpunk with Neon Organic Fabrics")

# Step 2: Copywriting
copy_result = copywriter_agent(market_research_result)

# Step 3: Packaging 
final_file = packaging_agent(
    trend_summary=market_research_result["trend_summary"],
    image_url="https://via.placeholder.com/800x400.png?text=Futuristic+Fashion+Concept", 
    quote=copy_result["quote"],
    justification=copy_result["justification"],
    output_path=f"campaign_summary_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.md"
)

print(f"\n Success! Report saved to: {final_file}")
display(Markdown(open(final_file).read()))

Packaging Agent rewriting for executives...

 Success! Report saved to: campaign_summary_2026-02-03_12-31-24.md


# Futuristic Clothing Campaign – Executive Summary

## Refined Trend Insights
Sustainable Cyberpunk with Neon Organic Fabrics redefines futuristic fashion by uniting cyberpunk’s bold, high-tech aesthetic with rigorous eco-credentials. At its core are biodegradable textiles—organic cotton, bamboo and recycled polyester—infused with vibrant, naturally sourced neon dyes in electric blue, shocking pink and acid green. Sharp, architectural cuts meet fluid silhouettes, offering both the structured edge and dynamic movement that cyberpunk demands. Advanced production methods—from water-free dye processes to 3D printing—ensure minimal waste and a reduced carbon footprint, without compromising on durability or comfort. For forward-thinking brands, this trend delivers a powerful narrative: cutting-edge design married to genuine sustainability, positioning your organization at the forefront of both environmental stewardship and next-generation style.

## Campaign Visual
![Campaign Visual](https://via.placeholder.com/800x400.png?text=Futuristic+Fashion+Concept)

## Campaign Quote
> Illuminate the Future: Where Eco-Conscious Meets Cyberpunk Chic.

## Why This Works
This slogan resonates with the target audience by capturing the dual essence of sustainability and the edgy, futuristic vibe of cyberpunk fashion. The word 'Illuminate' highlights the vivid neon colors that are integral to this trend, evoking a sense of excitement and innovation. 'Eco-Conscious' appeals to environmentally aware consumers who prioritize sustainable choices. 'Cyberpunk Chic' not only conveys the style's cutting-edge aesthetic but also reassures the audience that they can look stylish while supporting eco-friendly practices. Together, these elements speak to a fashion-forward audience interested in bold, ethical, and technologically advanced clothing.

---
*Report generated on 2026-02-03*